In [2]:
import tensorflow as tf
import glob 
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder
from numpy import array
from datetime import datetime

In [58]:
path=r'C:\Users\Joshlin\Downloads\infections-master\infections-master\warehouse\design\raw'

data_2 = pd.DataFrame()
def read_c(csv):
    testing_2 = pd.read_csv(csv)
    testing_2['filename'] = os.path.basename(csv).split('.')[0]
    return testing_2
data_2 = pd.concat(map(read_c, glob.glob(os.path.join(path, "*.csv"))))

In [59]:
data =  data_2
data.columns

Index(['date', 'covidOccupiedBeds', 'covidOccupiedMVBeds',
       'estimatedNewAdmissions', '0-4', '10-14', '15-19', '20-24', '25-29',
       '30-34', '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64',
       '65-69', '70-74', '75-79', '80-84', '85-89', '90+', 'dailyCases',
       'dailyFirstDoseByVaccinationDate', 'dailySecondDoseByVaccinationDate',
       'filename'],
      dtype='object')

In [29]:
data['date'] = pd.to_datetime(data['date'],format='%Y-%m-%d')
 #data =  data[data.date > '2020-12-31']
data['date'] = data['date'].astype('str')
data['date'] = data['date'].str.replace('-','')
print(data['date'])

0      20200301
1      20200302
2      20200303
3      20200304
4      20200305
         ...   
657    20211218
658    20211219
659    20211220
660    20211221
661    20211222
Name: date, Length: 92680, dtype: object


In [62]:
import numpy as np
data = data.sort_values(by=['filename','date'])
data['date'] = pd.to_datetime(data['date'])
data['new_date'] = (data['date'] - datetime(1970,1,1))
data['new_date'] = ( data['new_date'] / np.timedelta64(1, 'D')).astype(int)
column_name = ['covidOccupiedBeds','estimatedNewAdmissions','covidOccupiedMVBeds', '0-4', '10-14', '15-19', '20-24', '25-29',
       '30-34', '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64',
       '65-69', '70-74', '75-79', '80-84', '85-89', '90+', 'dailyCases',
       'dailyFirstDoseByVaccinationDate', 'dailySecondDoseByVaccinationDate','new_date']

scaler =StandardScaler()
data[column_name]=scaler.fit_transform(data[column_name])
le = LabelEncoder()
data['filename'] = le.fit_transform(data['filename'])
input_data = pd.DataFrame(data.drop('date',axis=1))
new_data = input_data
n_features = len(new_data.columns)
new_data.head()
final_data = new_data.values
final_data

array([[-6.39325676e-01, -5.19152760e-01, -5.76398887e-01, ...,
        -4.58394529e-01,  0.00000000e+00, -1.72943639e+00],
       [-6.39325676e-01, -5.19152760e-01, -5.76398887e-01, ...,
        -4.58394529e-01,  0.00000000e+00, -1.72420360e+00],
       [-6.39325676e-01, -5.19152760e-01, -5.76398887e-01, ...,
        -4.58394529e-01,  0.00000000e+00, -1.71897081e+00],
       ...,
       [ 5.74732511e-01,  1.19454245e+00, -5.76398887e-01, ...,
        -1.83599762e-01,  1.39000000e+02,  1.71897081e+00],
       [ 5.74732511e-01,  1.19454245e+00, -5.76398887e-01, ...,
        -1.37659931e-01,  1.39000000e+02,  1.72420360e+00],
       [-6.39325676e-01, -5.19152760e-01, -5.76398887e-01, ...,
        -9.47730591e-02,  1.39000000e+02,  1.72943639e+00]])

In [63]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix-1, 2]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [66]:
n_steps  = 7
X,y = split_sequences(final_data, n_steps)
X= np.array(X)
y =np.array(y)
val_limit = round((len(X)/100)*80)
val_2_limit = round((len(X)/100)*10) + val_limit
X_train  =  X[:val_limit]
y_train  = y[:val_limit]
X_val = X[val_limit:val_2_limit]
y_val = y[val_limit:val_2_limit]
X_test = X[val_2_limit:]
y_test = y[val_2_limit:]
n_features = X_train.shape[2]
print(X_train.shape)

(74139, 7, 27)


In [80]:
from keras.models import Sequential
from keras.layers import LSTM,GRU,SimpleRNN
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from keras import initializers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

In [81]:
initializer = tf.keras.initializers.HeUniform()
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True,input_shape=(n_steps, n_features),kernel_initializer=initializer))
model.add(LSTM(20,activation='relu',return_sequences=True))
model.add(Dense(1,'relu'))
model.compile(optimizer= Adam(learning_rate= 0.001),loss=tf.keras.losses.MeanSquaredLogarithmicError())
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 7, 50)             15600     
                                                                 
 lstm_23 (LSTM)              (None, 7, 20)             5680      
                                                                 
 dense_11 (Dense)            (None, 7, 1)              21        
                                                                 
Total params: 21,301
Trainable params: 21,301
Non-trainable params: 0
_________________________________________________________________


In [82]:
%%time
import numpy as np
#X_train = X_train.astype('float32')
#X_train = 
#y_train = np.asarray(y_train).astype('float32')
#X_val = np.asarray(X_val).astype('float32')
#y_val = np.asarray(y_val).astype('float32')
#print(X_train[0])


es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

tb = TensorBoard('logs')


model.fit(X_train, y_train,validation_data =(X_val,y_val),callbacks=[es, rlr, mcp, tb],epochs=200,verbose=1,shuffle=False)

Epoch 1/200
2317/2317 [==============================] - ETA: 0s - loss: 0.0819
Epoch 00001: val_loss improved from inf to 0.06390, saving model to weights.h5
2317/2317 [==============================] - 21s 8ms/step - loss: 0.0819 - val_loss: 0.0639 - lr: 0.0010
Epoch 2/200
2312/2317 [============================>.] - ETA: 0s - loss: 0.0630
Epoch 00002: val_loss did not improve from 0.06390
2317/2317 [==============================] - 18s 8ms/step - loss: 0.0631 - val_loss: 0.0701 - lr: 0.0010
Epoch 3/200
2310/2317 [============================>.] - ETA: 0s - loss: 0.0582
Epoch 00003: val_loss did not improve from 0.06390
2317/2317 [==============================] - 18s 8ms/step - loss: 0.0582 - val_loss: 0.0657 - lr: 0.0010
Epoch 4/200
2315/2317 [============================>.] - ETA: 0s - loss: 0.0563
Epoch 00004: val_loss improved from 0.06390 to 0.06164, saving model to weights.h5
2317/2317 [==============================] - 18s 8ms/step - loss: 0.0563 - val_loss: 0.0616 - lr: 0.0

In [84]:
import numpy as np

model_2 = Sequential() 
model_2.add(GRU(40,return_sequences=True,input_shape=(n_steps, n_features),activation='relu',kernel_initializer=initializer))
model_2.add(GRU(30,return_sequences=True, activation='relu'))
model_2.add(GRU(10,activation='relu',return_sequences=True))
model_2.add(Dense(1,activation='relu'))
model_2.summary()
model_2.compile(optimizer= Adam(learning_rate= 0.001),loss= tf.keras.losses.MeanSquaredLogarithmicError()) 
model_2.fit(X_train,y_train,validation_data =(X_val,y_val),epochs=25,verbose=1,shuffle=False,batch_size=662)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 7, 40)             8280      
                                                                 
 gru_4 (GRU)                 (None, 7, 30)             6480      
                                                                 
 gru_5 (GRU)                 (None, 7, 10)             1260      
                                                                 
 dense_13 (Dense)            (None, 7, 1)              11        
                                                                 
Total params: 16,031
Trainable params: 16,031
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
112/112 [==============================] - 9s 46ms/step - loss: 0.1355 - val_loss: 0.1475
Epoch 2/25
112/112 [==============================] - 5s 41ms/step - loss: 0.131

In [17]:
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1,activation='relu'))
model.compile(optimizer=Adam(learning_rate=0.02), loss='mean_absolute_error')
# fit model
model.fit(X_train, y_train, epochs=7000, verbose=1)

ValueError: Exception encountered when calling layer "conv1d" (type Conv1D).

Negative dimension size caused by subtracting 2 from 1 for '{{node conv1d/Conv1D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d/Conv1D/ExpandDims, conv1d/Conv1D/ExpandDims_1)' with input shapes: [?,1,1,26], [1,2,26,64].

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 1, 26), dtype=float32)

In [27]:
model_3 = Sequential()
model_3.add(Dense(200,activation='tanh',kernel_initializer=initializer,input_shape=(n_steps, n_features)))
model_3.add(Dense(100,activation='tanh'))
model_3.add(Dense(50,activation='tanh'))
model_3.add(Dense(1,activation='relu'))
model_3.compile(optimizer=Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredLogarithmicError())
# fit model
model_3.fit(X_train, y_train, epochs=7000, verbose=1,batch_size=662,shuffle=False)

Epoch 1/7000
112/112 [==============================] - 2s 9ms/step - loss: 2.5221
Epoch 2/7000
112/112 [==============================] - 1s 10ms/step - loss: 2.5221
Epoch 3/7000
112/112 [==============================] - 1s 10ms/step - loss: 2.5221
Epoch 4/7000
112/112 [==============================] - 1s 10ms/step - loss: 2.5221
Epoch 5/7000
103/112 [==========================>...] - ETA: 0s - loss: 2.4491

KeyboardInterrupt: 